In [49]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, RandomForestRegressor
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, RandomizedSearchCV
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
%matplotlib inline

from nltk.tokenize import RegexpTokenizer
from nltk import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

#import warnings
#warnings.simplefilter(action='ignore', category=FutureWarning)

In [50]:
df = pd.read_csv('dataset/final_data.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [51]:
df.head()

,title,subreddit,author,subreddit_id,id,comments,combined_title_&_comment,subred_numeric,tokens,num_words,average_word_len,lems,stems,neg,neu,pos,compound,sent_score
0,Mulvaney acknowledges Ukraine aid was withheld...,Liberal,first52,t5_2qxt5,djdfhm,NaN,Mulvaney acknowledges Ukraine aid was withheld...,0,"['Mulvaney', 'acknowledges', 'Ukraine', 'aid',...",10,6.200000,Mulvaney acknowledges Ukraine aid wa withheld ...,mulvaney acknowledg ukrain aid wa withheld to ...,0.000,0.769,0.231,0.4019,1
1,"I have concerns about trump ""hosting"" the G7 a...",Liberal,djinnisequoia,t5_2qxt5,djdeoe,NaN,"I have concerns about trump ""hosting"" the G7 a...",0,"['I', 'have', 'concerns', 'about', 'trump', 'h...",14,4.357143,I have concern about trump hosting the G7 at D...,I have concern about trump host the G7 at dora...,0.000,1.000,0.000,0.0000,0
2,Swing district Democrats build on cash advanta...,Liberal,Bobsind,t5_2qxt5,djbsw2,NaN,Swing district Democrats build on cash advanta...,0,"['Swing', 'district', 'Democrats', 'build', 'o...",10,6.100000,Swing district Democrats build on cash advanta...,swing district democrat build on cash advantag...,0.000,0.818,0.182,0.2500,1
3,Climate crisis will not be discussed at G7 nex...,Liberal,spaceghoti,t5_2qxt5,djay3y,NaN,Climate crisis will not be discussed at G7 nex...,0,"['Climate', 'crisis', 'will', 'not', 'be', 'di...",13,4.615385,Climate crisis will not be discussed at G7 nex...,climat crisi will not be discuss at G7 next ye...,0.255,0.745,0.000,-0.6249,-1
4,California cannabis king linked to Ukrainian w...,Liberal,StardustSpinner,t5_2qxt5,djaxo7,Here is another story about the same character...,California cannabis king linked to Ukrainian w...,0,"['California', 'cannabis', 'king', 'linked', '...",68,5.000000,California cannabis king linked to Ukrainian w...,california cannabi king link to ukrainian who ...,0.000,1.000,0.000,0.0000,0


## Defining X and y 

In [91]:
X = df['combined_title_&_comment']
y = df['subreddit']

## Train/Test Split 

In [92]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)

1738    10 Things You Never Knew About Orwell's 1984 |...
548     2020 Democrat field wants to expand union powe...
936     How extremist replacement theory became GOP an...
1389    United States President 2024, Donald Trump Jr....
1607    Hunter Biden says role with Ukraine firm was ‘...
                              ...                        
1130    Trump Smacks Down ABC News for 'Fake News' Foo...
1294    'Not one cent': Hunter Biden tells ABC News he...
860     Unsealed flight logs show that Trump was on Ep...
1459    Unrelated but cute.Blatant pandering for upvot...
1126                          For anyone who isn't aware.
Name: combined_title_&_comment, Length: 1500, dtype: object

In [168]:
pipe = Pipeline([
    ('tf', TfidfVectorizer()),
    ('lr', LogisticRegression())])

In [137]:
pipe_1 = Pipeline([
    ('cvec', CountVectorizer()),
    ('lr', LogisticRegression())
])

pipe_2 = Pipeline([
    ('tf', TfidfVectorizer()),
    ('lr', LogisticRegression())
])

pipe_3 = Pipeline([
    ('cvec', CountVectorizer()),
    ('rf', RandomForestClassifier())
    ])


In [138]:
pipe_2 =  Pipeline([
    ('tf', TfidfVectorizer()),
    ('lr', LogisticRegression())
])

In [139]:
pipe_3 = Pipeline([
    ('cvec', CountVectorizer()),
    ('rf', RandomForestClassifier())
    ])

In [161]:
pipe_params = {
    'cvec__max_features': [2500, 3000, 3500],
    'cvec__min_df': [2, 3],
    'cvec__max_df': [0.9, 0.95],
    'cvec__ngram_range': [(1,1), (1,2)],
    'lr__penalty' : ['l1', 'l2'],
    'lr__C': np.logspace(-4, 1, 4, 20),
    'lr__random_state': [40, 100]}

In [169]:
pipe_params = {
    'tf__max_features': [2500, 3000, 3500],
    'tf__min_df': [2, 3],
    'tf__max_df': [0.9, 0.95],
    'tf__ngram_range': [(1,1), (1,2)],
    'lr__penalty' : ['l1', 'l2'],
    'lr__C': np.logspace(-4, 1, 4, 20),
    'lr__random_state': [40, 100]
}

pipe_2_params = {
    'tf__max_features': [2500, 3000, 3500],
    'tf__min_df': [2, 3],
    'tf__max_df': [0.9, 0.95],
    'tf__ngram_range': [(1,1), (1,2)],
    'lr__penalty' : ['l1', 'l2'],
    'lr__C': np.logspace(-4, 1, 4, 20),
    'lr__random_state': [40, 100]
}

pipe_3_params = {
    'tf__max_features': [2500, 3000, 3500],
    'tf__min_df': [2, 3],
    'tf__max_df': [0.9, 0.95],
    'tf__ngram_range': [(1,1), (1,2)],}
    

In [170]:
gs_2 = GridSearchCV(pipe, # what object are we optimizing?
                  pipe_params, # what parameters values are we searching?
                 cv = 3 ) # 3-fold cross-validation.

In [171]:
gs_2.fit(X_train, y_train)

KeyboardInterrupt: 

In [167]:
gs.best_score_

0.688

## Setting Up the Pipeline 

In [98]:
pipe_1 = ([
    ('cvec', CountVectorizer()),
    ('lr', LogisticRegression())
])

pipe_2 = ([
    ('tf', TfidfVectorizer()),
    ('lr', LogisticRegression())
])

pipe_3 = ([
    ('cvec', CountVectorizer()),
    ('rf', RandomForestClassifier())
    ])

pipe_4 = ([
    ('tf', TfidfVectorizer()),
    ('rf', RandomForestClassifier())
])

pipe_5 = ([
    ('cvec', CountVectorizer()),
    ('et', ExtraTreesClassifier())
])

pipe_6 = ([
    ('tf', TfidfVectorizer()),
    ('et', ExtraTreesClassifier())
])


pipe_7 =([
    ('cvec', CountVectorizer()),
    ('bag', BaggingClassifier())
])

pipe_8 =([
    ('tf', TfidfVectorizer()),
    ('bag', BaggingClassifier())
])

pipe_9 =  ([
    ('cvec', CountVectorizer()),
    ('svc', SVC(gamma= 'scale'))
])

pipe_10 = ([
    ('tf', TfidfVectorizer()),
    ('svc', SVC(gamma = 'scale'))
])

## Hyperparameters 

In [102]:
pipe_1_params = {
    'cvec__max_features': [2500, 3000, 3500],
    'cvec__min_df': [2,3],
    'cvec__max_df': [0.9, 0.95], 
    'cvec__ngram_range': [(1,1), (1,2)],
    'lr__penalty' : ['l1', 'l2'],
    'lr__C': np.logspace(-4, 1, 4, 20),
    'lr__random_state': [40, 100]
}

pipe_2_params = {
    'tf__max_features': [2500, 3000, 3500],
    'tf__min_df': [2,3],
    'tf__max_df': [0.9, 0.95], 
    'tf__ngram_range': [(1,1), (1,2)],
    'lr__penalty' : ['l1', 'l2'],
    'lr__C': np.logspace(-4, 1, 4, 20),
    'lr__random_state': [40, 50, 100]
}

pipe_3_params =   {
    'cvec__max_features': [2500, 3000, 3500],
    'cvec__min_df': [2,3],
    'cvec__max_df': [0.9, 0.95], 
    'cvec__ngram_range': [(1,1), (1,2)],
    'rf__bootstrap' : [True, False],
    'rf__max_depth' : [10, 50, 100],
    'rf__min_samples_leaf': [1,2,4],
    'rf__min_samples_split': [2, 5, 10], 
    'rf__n_estimators': [200, 500, 1000]
}

pipe_4_params = {
    'tf__max_features': [2500, 3000, 3500],
    'tf__min_df': [2,3],
    'tf__max_df': [0.9, 0.95], 
    'tf__ngram_range': [(1,1), (1,2)],
    'rf__bootstrap' : [True, False],
    'rf__max_depth' : [10 50, 100],
    'rf__min_samples_leaf': [1,2,4],
    'rf__min_samples_split': [2, 5, 10], 
    'rf__n_estimators': [200, 500, 1000]}

pipe_5_params = {
    'cvec__max_features': [2500, 3000, 3500],
    'cvec__min_df': [2,3],
    'cvec__max_df': [0.9, 0.95], 
    'cvec__ngram_range': [(1,1), (1,2)],
    'et__max_depth' : [10,20,30, 50, 100],
    'et__min_samples_leaf': [1,2,4],
    'et__min_samples_split': [2, 5, 10], 
    'et__n_estimators': [200, 500, 800, 1000]}

pipe_6_params = {
    'tf__max_features': [2500, 3000, 3500],
    'tf__min_df': [2,3],
    'tf__max_df': [0.9, 0.95], 
    'tf__ngram_range': [(1,1), (1,2)],
    'et__max_depth' : [10,20,30, 50, 100],
    'et__min_samples_leaf': [1,2,4],
    'et__min_samples_split': [2, 5, 10], 
    'et__n_estimators': [200, 500, 800, 1000]}

pipe_7_params = {
    'cvec__max_features': [2500, 3000, 3500],
    'cvec__min_df': [2,3],
    'cvec__max_df': [0.9, 0.95], 
    'cvec__ngram_range': [(1,1), (1,2)],
    'bag__max_features' : [100, 200],
    'bag__bootstrap': [True, False]}

pipe_8_params = {
    'tf__max_features': [2500, 3000, 3500],
    'tf__min_df': [2,3],
    'tf__max_df': [0.9, 0.95], 
    'tf__ngram_range': [(1,1), (1,2)],
    'bag__max_features' : [100, 200],
    'bag__bootstrap': [True, False]}

pipe_9_params = {
    'cvec__max_features': [2500, 3000, 3500],
    'cvec__min_df': [2,3],
    'cvec__max_df': [0.9, 0.95], 
    'cvec__ngram_range': [(1,1), (1,2)],
    'svc__kernel': ['linear', 'rbf', 'poly'],
    'svc__C': np.logspace(-1, 2, 30)
}

pipe_10_params = {
    'tf__max_features': [2500, 3000, 3500],
    'tf__min_df': [2,3],
    'tf__max_df': [0.9, 0.95], 
    'tf__ngram_range': [(1,1), (1,2)],
    'svc__kernel': ['linear', 'rbf', 'poly'],
    'svc__C': np.logspace(-1, 2, 30)
    
}
    
    


## Grid Search

In [172]:
grid_1 = GridSearchCV(pipe_1, pipe_1_params, cv =3, verbose =1  )

In [173]:
grid_1.fit(X_train, y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed:   20.8s finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('cvec', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'cvec__max_features': [2500, 3000, 3500], 'cvec__min_df': [2, 3], 'cvec__max_df': [0.9, 0.95], 'cvec__ngram_range': [(1, 1), (1, 2)]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [105]:
pipe_1_parms_test = {
    #'cvec__max_features': [2500, 3000, 3500],
    #'cvec__min_df': [2,3],
    #'cvec__max_df': [0.9, 0.95], 
    #'cvec__ngram_range': [(1,1), (1,2)],
    'penalty' : ['l1', 'l2'],
    'C': np.logspace(-4, 1, 4, 20),
    'random_state': [40, 100]
}